In [22]:
import pandas as pd
from pathlib import Path

BASE_2017 = Path(".")   # carpeta actual

print("CWD:", Path.cwd())
print("Archivos:", [p.name for p in BASE_2017.iterdir()])




CWD: c:\Users\nuez_\Desktop\Proyecto ENSE\Hospitales_raw\Hospitales_2017
Archivos: ['05_Hospitales2017.ipynb', 'CH_Catalogo_2017.csv', 'CH_CCAA_2017.csv', 'CH_Patrimonial_2017.csv', 'CH_Provincias_2017.csv']


In [23]:
from pathlib import Path

p = Path("CH_Catalogo_2017.csv")

# leer unas pocas líneas como bytes (no falla por encoding)
raw = p.read_bytes()[:2000]
print(raw[:200])  # vistazo rápido

# ahora como texto (latin1 aguanta casi todo)
txt = raw.decode("latin1", errors="replace")
print("----LINEA 1----")
print(txt.splitlines()[0])
print("----LINEA 2----")
print(txt.splitlines()[1])

# cuenta separadores típicos en la cabecera
h = txt.splitlines()[0]
print("count(';') =", h.count(";"))
print("count(',') =", h.count(","))
print("count('\\t') =", h.count("\t"))


b'CODCNH;NOMBRE;DIRECCION;TELEFONO;TELEFONO2;TELEFAX;CODMU;CODPROV;CODAUTO;CODPOSTAL;NCAMAS;CODFI;CODPAT;CODFU;ACREDOCENT;ESCOMPLE;FORCOMPLE;CODIDCOM;ALTA;CERRADO;CAPITAL;CIERREFECH;CONCIERTO;EMAIL\r\n100'
----LINEA 1----
CODCNH;NOMBRE;DIRECCION;TELEFONO;TELEFONO2;TELEFAX;CODMU;CODPROV;CODAUTO;CODPOSTAL;NCAMAS;CODFI;CODPAT;CODFU;ACREDOCENT;ESCOMPLE;FORCOMPLE;CODIDCOM;ALTA;CERRADO;CAPITAL;CIERREFECH;CONCIERTO;EMAIL
----LINEA 2----
10035;RED DE SALUD MENTAL DE ARABA (HOSPITAL PSIQUIÃTRICO DE ARABA);Ãlava, 43;945006555;;945006587;10590;1;16;1006;207;6;6;4;N;N;N;;N;N;S;;N;SECRETARIA.GERENCIA@osakidetza.eus
count(';') = 23
count(',') = 0
count('\t') = 0


In [24]:
import pandas as pd
from pathlib import Path

BASE_2017 = Path(".")  # carpeta actual: Hospitales_raw/Hospitales_2017

cat_path  = next(BASE_2017.glob("CH_Cat*.csv"))
ccaa_path = next(BASE_2017.glob("CH_CCAA*.csv"))
pat_path  = next(BASE_2017.glob("CH_Pat*.csv"))

cat  = pd.read_csv(cat_path,  sep=";", encoding="latin1", low_memory=False)
ccaa = pd.read_csv(ccaa_path, sep=";", encoding="latin1", low_memory=False)
pat  = pd.read_csv(pat_path,  sep=";", encoding="latin1", low_memory=False)

print("shapes:", cat.shape, ccaa.shape, pat.shape)


print("\nCAT columnas clave presentes?:", [c for c in ["CODAUTO","CODPAT","CODFU"] if c in cat.columns])
print("CCAA columnas:", ccaa.columns.tolist())
print("PAT columnas:", pat.columns.tolist())

cat[["CODAUTO","CODPAT","CODFU"]].head()




shapes: (888, 24) (19, 3) (15, 2)

CAT columnas clave presentes?: ['CODAUTO', 'CODPAT', 'CODFU']
CCAA columnas: ['AUTONOMIA', 'CODAUTO', 'TEXTO_AUTONOMIA']
PAT columnas: ['CODPAT', 'DESDE']


,CODAUTO,CODPAT,CODFU
0,16,6,4
1,16,14,23
2,16,14,23
3,16,14,23
4,16,6,4


In [52]:
cat.head()



,CODCNH,NOMBRE,DIRECCION,TELEFONO,TELEFONO2,TELEFAX,CODMU,CODPROV,CODAUTO,CODPOSTAL,...,FORCOMPLE,CODIDCOM,ALTA,CERRADO,CAPITAL,CIERREFECH,CONCIERTO,EMAIL,CCAA_nombre,Dependencia_patrimonial
0,10035,RED DE SALUD MENTAL DE ARABA (HOSPITAL PSIQUIÃ...,"Ãlava, 43",945006555.0,NaN,945006587.0,10590,1,16,1006,...,N,NaN,N,N,S,NaN,N,SECRETARIA.GERENCIA@osakidetza.eus,ILLES BALEARS,COMUNIDAD AUTÓNOMA
1,10040,HOSPITAL SAN JOSÃ,"Beato TomÃ¡s de Zumarraga, 10",945140900.0,NaN,945145709.0,10590,1,16,1008,...,N,NaN,N,N,S,NaN,N,diazdecorcueram@vithas.es,NaN,PRIVADO NO BENÉFICO
2,10053,HOSPITAL QUIRÃNSALUD VITORIA,"Esperanza, 3",945252500.0,NaN,945279260.0,10590,1,16,1002,...,N,NaN,N,N,S,NaN,S,josecarlos.mellado@quironsalud.es,NaN,PRIVADO NO BENÉFICO
3,10066,"HOSPITAL DE CUIDADOS SAN ONOFRE, S.L. (HOSPITA...","Salvatierrabide, 9",945142100.0,NaN,945143318.0,10590,1,16,1007,...,N,NaN,N,N,S,NaN,S,info@hospitalsanonofre.com,NaN,PRIVADO NO BENÉFICO
4,10088,HOSPITAL DE LEZA,"Ctra. Vitoria - LogroÃ±o, km. 59",945006900.0,945006876.0,945006901.0,10318,1,16,1309,...,N,NaN,N,N,N,NaN,N,hospital.leza@osakidetza.eus,ILLES BALEARS,COMUNIDAD AUTÓNOMA


In [54]:
ccaa = ccaa.rename(columns={
    "AUTONOMIA": "CODAUTO",
    "CODAUTO": "CCAA_nombre"
})


In [57]:
ccaa.head()
ccaa.dtypes

CODAUTO            object
CCAA_nombre        object
TEXTO_AUTONOMIA    object
dtype: object

In [58]:
ccaa["CODAUTO"] = (
    ccaa["CODAUTO"]
      .astype(str)
      .str.strip()
      .str.replace(r"\D", "", regex=True)  # por si viniera "01"
)

ccaa["CODAUTO"] = pd.to_numeric(ccaa["CODAUTO"], errors="coerce").astype("Int64")


In [59]:
ccaa["CODAUTO"].head(10)
ccaa["CODAUTO"].unique()
ccaa["CODAUTO"].isna().sum()


np.int64(0)

In [60]:
cat["CODAUTO"] = pd.to_numeric(cat["CODAUTO"], errors="coerce").astype("Int64")


In [61]:
cat["CODAUTO"].unique()
cat["CODAUTO"].isna().sum()


np.int64(0)

In [28]:
pat.head()

,CODPAT,DESDE
0,1,SEGURIDAD SOCIAL
1,2,INSTITUTO DE SALUD CARLOS III
2,3,MINISTERIO DE EDUCACIÓN Y CIENCIA
3,4,MINISTERIO DE INTERIOR
4,5,MINISTERIO DE DEFENSA


In [62]:
cat = cat.merge(
    ccaa[["CODAUTO", "CCAA_nombre"]],
    on="CODAUTO",
    how="left"
)


In [67]:
cat.tail()

,CODCNH,NOMBRE,DIRECCION,TELEFONO,TELEFONO2,TELEFAX,CODMU,CODPROV,CODAUTO,CODPOSTAL,...,CODIDCOM,ALTA,CERRADO,CAPITAL,CIERREFECH,CONCIERTO,EMAIL,CCAA_nombre_x,Dependencia_patrimonial,CCAA_nombre_y
883,500195,PROYECTO DE REHAB. INTEGRAL DE LA SALUD MENTAL...,"Torre del cerezo, nÂº 17 BÂº. de San Juan de M...",976151152.0,NaN,976151153.0,502973,50,2,50820,...,NaN,N,N,S,NaN,N,administracion@hospitalprisma.com,NaN,PRIVADO NO BENÉFICO,ARAGÓN
884,500200,CENTRO SANITARIO CINCO VILLAS,"Antonio Machado, s/n",976677978.0,NaN,976677972.0,500956,50,2,50600,...,NaN,N,N,N,NaN,N,NaN,NaN,COMUNIDAD AUTÓNOMA,ARAGÓN
885,500218,"CLÃNICA MONTECANAL, SL","C/ Franz Shubert, 2",876241818.0,NaN,876241822.0,502973,50,2,50012,...,NaN,N,N,S,NaN,S,admi@viamedmontecanal.com,NaN,PRIVADO NO BENÉFICO,ARAGÓN
886,510039,HOSPITAL UNIVERSITARIO DE CEUTA,"Loma Colmenar, s/n",856907000.0,NaN,856907066.0,510013,51,18,51003,...,NaN,N,N,S,NaN,N,NaN,ANDALUCÍA,SEGURIDAD SOCIAL,CEUTA
887,520018,HOSPITAL COMARCAL,"Ctra. Remonta, 2",952698001.0,NaN,952698032.0,520018,52,19,52005,...,NaN,N,N,S,NaN,N,inform@hcml.insalud.es,ANDALUCÍA,SEGURIDAD SOCIAL,MELILLA


In [68]:
cat["CCAA"] = cat["CCAA_nombre_y"]

cat = cat.drop(columns=["CCAA_nombre_x", "CCAA_nombre_y"])


In [72]:
cat.head()

,CODCNH,NOMBRE,DIRECCION,TELEFONO,TELEFONO2,TELEFAX,CODMU,CODPROV,CODAUTO,CODPOSTAL,...,FORCOMPLE,CODIDCOM,ALTA,CERRADO,CAPITAL,CIERREFECH,CONCIERTO,EMAIL,Dependencia_patrimonial,CCAA
0,10035,RED DE SALUD MENTAL DE ARABA (HOSPITAL PSIQUIÃ...,"Ãlava, 43",945006555.0,NaN,945006587.0,10590,1,16,1006,...,N,NaN,N,N,S,NaN,N,SECRETARIA.GERENCIA@osakidetza.eus,COMUNIDAD AUTÓNOMA,PAÍS VASCO
1,10040,HOSPITAL SAN JOSÃ,"Beato TomÃ¡s de Zumarraga, 10",945140900.0,NaN,945145709.0,10590,1,16,1008,...,N,NaN,N,N,S,NaN,N,diazdecorcueram@vithas.es,PRIVADO NO BENÉFICO,PAÍS VASCO
2,10053,HOSPITAL QUIRÃNSALUD VITORIA,"Esperanza, 3",945252500.0,NaN,945279260.0,10590,1,16,1002,...,N,NaN,N,N,S,NaN,S,josecarlos.mellado@quironsalud.es,PRIVADO NO BENÉFICO,PAÍS VASCO
3,10066,"HOSPITAL DE CUIDADOS SAN ONOFRE, S.L. (HOSPITA...","Salvatierrabide, 9",945142100.0,NaN,945143318.0,10590,1,16,1007,...,N,NaN,N,N,S,NaN,S,info@hospitalsanonofre.com,PRIVADO NO BENÉFICO,PAÍS VASCO
4,10088,HOSPITAL DE LEZA,"Ctra. Vitoria - LogroÃ±o, km. 59",945006900.0,945006876.0,945006901.0,10318,1,16,1309,...,N,NaN,N,N,N,NaN,N,hospital.leza@osakidetza.eus,COMUNIDAD AUTÓNOMA,PAÍS VASCO


In [71]:
cat[["CODAUTO", "CCAA"]].head(10)
cat["CCAA"].isna().sum(), len(cat)


(np.int64(0), 888)

In [73]:
cat["año"] = 2017


In [ ]:
cat["Dependencia_patrimonial"].value_counts(dropna=False).head(30)


In [76]:
import numpy as np

publicos = [
    "COMUNIDAD AUTÓNOMA",
    "SEGURIDAD SOCIAL",
    "ENTIDADES PÚBLICAS",
    "DIPUTACIÓN O CABILDO",
    "MUNICIPIO",
    "MINISTERIO DE DEFENSA",
    "MINISTERIO DE INTERIOR",
]

privados = [
    "PRIVADO NO BENÉFICO",
    "OTRO PRIVADO BENÉFICO",
    "PRIVADO-BENÉFICO (IGLESIA)",
    "PRIVADO-BENÉFICO (CRUZ ROJA)",
]

cat["titularidad"] = np.select(
    [
        cat["Dependencia_patrimonial"].isin(publicos),
        cat["Dependencia_patrimonial"].isin(privados),
    ],
    [
        "Público",
        "Privado",
    ],
    default="Otros"
)


In [77]:
cat["titularidad"].value_counts(dropna=False)


titularidad
Privado    435
Público    434
Otros       19
Name: count, dtype: int64

In [51]:
cat["CCAA_nombre"].isna().sum(), len(cat)


(np.int64(611), 888)

In [ ]:
cat.head()

In [81]:
# me quedo con lo mínimo
base_2017 = cat[["CCAA", "año", "titularidad"]].copy()

# sanity check
base_2017["titularidad"].value_counts()
base_2017["CCAA"].isna().sum(), len(base_2017)


(np.int64(0), 888)

In [82]:
base_2017["titularidad"].value_counts()

titularidad
Privado    435
Público    434
Otros       19
Name: count, dtype: int64

In [83]:
base_2017.head()

,CCAA,año,titularidad
0,PAÍS VASCO,2017,Público
1,PAÍS VASCO,2017,Privado
2,PAÍS VASCO,2017,Privado
3,PAÍS VASCO,2017,Privado
4,PAÍS VASCO,2017,Público


In [84]:
hosp_2017_long = (
    base_2017
    .groupby(["CCAA", "año", "titularidad"])
    .size()
    .reset_index(name="n_hospitales")
)

hosp_2017_long.head(10)


,CCAA,año,titularidad,n_hospitales
0,ANDALUCÍA,2017,Otros,1
1,ANDALUCÍA,2017,Privado,61
2,ANDALUCÍA,2017,Público,70
3,ARAGÓN,2017,Otros,1
4,ARAGÓN,2017,Privado,8
5,ARAGÓN,2017,Público,20
6,C. FORAL DE NAVARRA,2017,Privado,7
7,C. FORAL DE NAVARRA,2017,Público,4
8,CANARIAS,2017,Privado,22
9,CANARIAS,2017,Público,26


In [85]:
hosp_2017_long["n_hospitales"].sum()


np.int64(888)

In [90]:
# 1) Cuenta hospitales por CCAA y titularidad
hosp_2017 = (
    base_2017
    .groupby(["CCAA", "año", "titularidad"], dropna=False)
    .size()
    .reset_index(name="n_hospitales")
)

# 2) Total de hospitales por CCAA (todas las titularidades)
total_2017 = (
    base_2017
    .groupby(["CCAA", "año"])
    .size()
    .reset_index(name="total_hospitales")
)

# 3) Unimos total para poder calcular porcentajes
hosp_2017 = hosp_2017.merge(total_2017, on=["CCAA", "año"], how="left")
hosp_2017["pct"] = (hosp_2017["n_hospitales"] / hosp_2017["total_hospitales"]) * 100

hosp_2017.head(10)


,CCAA,año,titularidad,n_hospitales,total_hospitales,pct
0,ANDALUCÍA,2017,Otros,1,132,0.757576
1,ANDALUCÍA,2017,Privado,61,132,46.212121
2,ANDALUCÍA,2017,Público,70,132,53.030303
3,ARAGÓN,2017,Otros,1,29,3.448276
4,ARAGÓN,2017,Privado,8,29,27.586207
5,ARAGÓN,2017,Público,20,29,68.965517
6,C. FORAL DE NAVARRA,2017,Privado,7,11,63.636364
7,C. FORAL DE NAVARRA,2017,Público,4,11,36.363636
8,CANARIAS,2017,Privado,22,48,45.833333
9,CANARIAS,2017,Público,26,48,54.166667


In [91]:
hosp_2017["pct"] = hosp_2017["pct"].round(2)


In [94]:
hosp_2017

,CCAA,año,titularidad,n_hospitales,total_hospitales,pct
0,ANDALUCÍA,2017,Otros,1,132,0.76
1,ANDALUCÍA,2017,Privado,61,132,46.21
2,ANDALUCÍA,2017,Público,70,132,53.03
3,ARAGÓN,2017,Otros,1,29,3.45
4,ARAGÓN,2017,Privado,8,29,27.59
5,ARAGÓN,2017,Público,20,29,68.97
6,C. FORAL DE NAVARRA,2017,Privado,7,11,63.64
7,C. FORAL DE NAVARRA,2017,Público,4,11,36.36
8,CANARIAS,2017,Privado,22,48,45.83
9,CANARIAS,2017,Público,26,48,54.17


In [ ]:
(
    hosp_2017
    .groupby(["CCAA", "año"])["pct"]
    .sum()
    .round(2)
    .sort_values(ascending=False)
)


In [95]:
hosp_2017.to_csv(
    "Hospitales_agregado_2017.csv",
    index=False,
    encoding="utf-8-sig"
)


In [96]:
import os
os.getcwd()


'c:\\Users\\nuez_\\Desktop\\Proyecto ENSE\\Hospitales_raw\\Hospitales_2017'

In [97]:
os.listdir()


['05_Hospitales2017.ipynb',
 'CH_Catalogo_2017.csv',
 'CH_CCAA_2017.csv',
 'CH_Patrimonial_2017.csv',
 'CH_Provincias_2017.csv',
 'Hospitales_agregado_2017.csv']